In [ ]:
import pandas as pd
import pickle
from tqdm import tqdm_notebook
from datetime import datetime, date, time
import numpy as np
pd.set_option('display.max_columns', 500)

In [ ]:
data = pd.read_excel(r'C:\Users\korpachev\Comments_Semantic_Analysis\Data_Comments.xlsx')

In [ ]:
from ufal.udpipe import Model, Pipeline
model = Model.load("russian-syntagrus-ud-2.0-170801.udpipe") 

In [ ]:
pipeline = Pipeline(model, 'generic_tokenizer', '', '', '')
example = "Очень комфортно взаимодействовать с менеджером"
parsed = pipeline.process(example)
parsed.split('\n')

# Анализ

In [ ]:
with open('treaten/Менеджер- Менеджер К.pickle', 'rb') as f:
    lemmas_list, lemmas_clear_list = pickle.load(f)
def elementFreq(myList):
    #myList is the list of lists
    from collections import Counter
    tmp = []
    for i in myList: tmp += i        
    return(Counter(tmp))
    
lemmas_clear_list = [e for i in lemmas_clear_list for e in i]
lemmas_clear_list = pd.DataFrame.from_dict(elementFreq(lemmas_clear_list).most_common())
l_noun = []
for j in list(lemmas_clear_list[0]):
    parsed = pipeline.process(j)
    v1 = 'NOT'
    for e in parsed.split('\n'):
        if e.split('\t')[0] == '1' and  e.split('\t')[3] == 'NOUN':
            v1 = 'NOUN'
    l_noun.append(v1)
lemmas_clear_list['noun'] = l_noun
lemmas_clear_list = lemmas_clear_list[lemmas_clear_list['noun'] == 'NOUN']

In [ ]:
lemmas_clear_list.head(15)

In [ ]:
def elementFreq(myList):
    #myList is the list of lists
    from collections import Counter
    tmp = []
    for i in myList: tmp += i        
    return(Counter(tmp))

In [ ]:
qlist

In [ ]:
df_list_main = []

qlist = [['Менеджер', ' Менеджер К'],['Личный кабинет', 'Личный кабинет К'],['Приложение', 'Приложение К'],['Офис', 'Офис К'],['Маркетинг', 'Маркетинг К'],['NPS','Что нам необходимо сделать, чтобы в следующий раз вы поставили оценку 10? Пожалуйста, оставьте комментарий']][0:-1]
for category in qlist[0:-1][0:1]:
    print(' '.join(u))
    u = category    
    q1 = data[data[u[0]] != 'Не было взаимодействия'][u]
    q1 = q1[q1[u[1]] != '  ']
    
    with open('treaten/{}.pickle'.format('-'.join(u)), 'rb'.format()) as f:
        lemmas_list, lemmas_clear_list = pickle.load(f)

    lemmas_clear_list = [e for i in lemmas_clear_list for e in i]
    lemmas_clear_list = pd.DataFrame.from_dict(elementFreq(lemmas_clear_list).most_common())
    l_noun = []
    for j in list(lemmas_clear_list[0]):
        parsed = pipeline.process(j)
        v1 = 'NOT'
        for e in parsed.split('\n'):
            if e.split('\t')[0] == '1' and  e.split('\t')[3] == 'NOUN':
                v1 = 'NOUN'
        l_noun.append(v1)
    lemmas_clear_list['noun'] = l_noun
    lemmas_clear_list = lemmas_clear_list[lemmas_clear_list['noun'] == 'NOUN']
    display(lemmas_clear_list.head(20))

    df_list = []
    
    for word_m in tqdm_notebook(lemmas_clear_list.head(15)[0]):
        print(word_m)
        word_main = word_m.lower()
        
        ep_fin = []
        e_fin = []
        sent_fin = []
        
        for row in list(q1[u[1]]):
            parsed = pipeline.process(str(row))
            ep_row = []
            e_row = []
            sent_row = []
            for p in parsed.split("sent_id")[1:]:

                full_sent = ' '.join([e for m in p.split("\n")[2:-2] for i,e in enumerate(m.split("\t")) if i in [1]])
                entities = [[e.split('\t')[0], e.split('\t')[2]] for e in p.split("\n")[2:-2] if e.split('\t')[3] == 'NOUN' and e.split('\t')[2].lower() == word_main]

                if entities != []: # Если внутри предложения в строчке нет entity - [] не добавляется
                    for e in entities:
                        entity_only= e[1].lower()
                        entity_phrase = e[1]
                        for m in p.split("\n")[2:-2]:
                            if m.split('\t')[6] == e[0]:
                                if m.split('\t')[3] == 'ADJ':
                                       entity_phrase = (entity_phrase + ' ' + m.split('\t')[2]).lower()
                    ep_row.append(entity_phrase)
                    e_row.append(entity_only)
                    sent_row.append(full_sent)
            ep_fin.append(ep_row)
            e_fin.append(e_row)
            sent_fin.append(sent_row)

        ep_fin = [e for i in ep_fin for e in i] 
        e_fin = [e for i in e_fin for e in i] 
        sent_fin = [e for i in sent_fin for e in i] 

        df_raw = pd.DataFrame({'entity_full': ep_fin, 'sentence': sent_fin })
        df = df_raw.groupby('entity_full')['sentence'].apply(list).to_frame().reset_index()
        df['count'] = df['sentence'].apply(lambda x: len(x))
        df.sort_values('count',ascending=False, inplace=True)
        df['sentence'] = df['sentence'].apply(lambda x: ' | '.join(x))
        df.rename(columns={'entity_full': 'Сущность + прилагательное', 'sentence': 'Фразы', 'count': 'Частота'}, inplace=True)

        df_0 = df[df['Сущность + прилагательное'] == word_main]
        df_0['agg'] = 1000000
        df_0['Основная сущность + прилагательное'] = df_0['Сущность + прилагательное']

        df_1 = df[df['Сущность + прилагательное'] != word_main]
        import heapq
        ler = list(df_1['Сущность + прилагательное'])
        l1 = [e.replace('менеджер','').strip() for e in ler if e.replace(word_main,'').strip() != '']

        stf= []
        for m in l1:
            stemp = []
            for e in data_syms:
                for i, j in enumerate(e):
                    if j == m:
                        stemp.append(e)
            stemp = list(set([k for z in stemp for k in z]))
            stf.append(stemp)

        t2=[]
        t2_indexes = []
        for e in stf:
            t1= []
            for m in stf:
                t1.append(len(list(set(e) & set(m))))
            t2.append(t1)
            max2 = heapq.nlargest(2, t1)[-1] 

            if max2 == 0:
                t2_indexes.append(1000000)
            else:
                t2_indexes.append(t1.index(max2))

        df_1['agg'] = t2_indexes
        df_2 = df_1[df_1['agg'] == 1000000]
        df_2['Основная сущность + прилагательное'] = df_2['Сущность + прилагательное']
        df_1 = df_1[df_1['agg'] != 1000000]
        df_1 = df_1.groupby('agg').agg({'Сущность + прилагательное' : list,'Фразы' : list, 'Частота': sum }).sort_values('Частота',ascending=False)
        df_1['agg'] = 1000000
        df_1['Основная сущность + прилагательное'] = df_1['Сущность + прилагательное'].apply(lambda x: x[0])
        df_1['Сущность + прилагательное'] = df_1['Сущность + прилагательное'].apply(lambda x: ' | '.join(x))
        df_1['Фразы'] = df_1['Фразы'].apply(lambda x: ' | '.join(x))
        df = pd.concat([df_0,df_1,df_2],axis=0)[['Основная сущность + прилагательное','Сущность + прилагательное','Фразы','Частота']]             
        df['Фильтр по медиане'] = np.where((df['Частота'] >= df['Частота'].median()), 'Выше медианы', 'Ниже медианы')
        df.reset_index(drop=True, inplace=True)
        #df.to_excel('treaten_sem_anal/{}.xlsx'.format(word_main),index=False)
        df[''] = ''
        print(df.shape)
        df_list.append(df)

        ep_fin = []
        e_fin = []
        sent_fin = []
        for row in list(q1[u[1]]):
            parsed = pipeline.process(str(row))
            ep_row = []
            e_row = []
            sent_row = []
            for p in parsed.split("sent_id")[1:]:
                full_sent = ' '.join([e for m in p.split("\n")[2:-2] for i,e in enumerate(m.split("\t")) if i in [1]])
                entities = [[e.split('\t')[6], e.split('\t')[2]] for e in p.split("\n")[2:-2] if e.split('\t')[3] == 'NOUN' and e.split('\t')[2].lower() == word_main]

                if entities != []: # Если внутри предложения в строчке нет entity - [] не добавляется
                    for e in entities:
                        entity_only= e[1].lower()
                        entity_phrase = e[1]
                        verb_form = ''
                        for m in p.split("\n")[2:-2]:
                            if m.split('\t')[0] == e[0]:
                                if m.split('\t')[3] == 'VERB':
                                    verb_id = m.split('\t')[0]
                                    verb_form = m.split('\t')[2]
                                    for j in p.split("\n")[2:-2]:
                                        if j.split('\t')[6] == verb_id:
                                            if j.split('\t')[7] == 'advmod':
                                                if j.split('\t')[3] == 'PART':
                                                    verb_form = j.split('\t')[2] + " " + verb_form
                                                else:
                                                    verb_form = verb_form + " " + j.split('\t')[2]
                                            else:
                                                if j.split('\t')[3] == 'NOUN' and j.split('\t')[2].lower() != word_main:

                                                    verb_form = verb_form + " " + j.split('\t')[2]

                        entity_phrase = (entity_phrase + " " + verb_form).lower()
                        ep_row.append(entity_phrase)
                        e_row.append(entity_only)
                        sent_row.append(full_sent)
            ep_fin.append(ep_row)
            e_fin.append(e_row)
            sent_fin.append(sent_row)
            
        ep_fin = [e for i in ep_fin for e in i] 
        e_fin = [e for i in e_fin for e in i] 
        sent_fin = [e for i in sent_fin for e in i] 

        df_raw = pd.DataFrame({'entity_full': ep_fin, 'sentence': sent_fin })
        df = df_raw.groupby('entity_full')['sentence'].apply(list).to_frame().reset_index()
        df['count'] = df['sentence'].apply(lambda x: len(x))
        df.sort_values('count',ascending=False, inplace=True)
        df['sentence'] = df['sentence'].apply(lambda x: ' | '.join(x))
        df.rename(columns={'entity_full': 'Сущность + глагол', 'sentence': 'Фразы', 'count': 'Частота'}, inplace=True)
        #    df = df[df['Частота'] > df['Частота'].median()]
        df['Фильтр по медиане'] = np.where((df['Частота'] >= df['Частота'].median()), 'Выше медианы', 'Ниже медианы')
        df.reset_index(drop=True, inplace=True)
        df[''] = ''
        print(df.shape)
        df_list.append(df)
    print('Добавляют все сущности по вопросам в категории')
    df_list_main.append(pd.concat(df_list,axis=1))